In [1]:
from model import EASE
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:

df = pd.read_csv('/opt/ml/input/code/data/train/train_ratings.csv')
df["time"] = 1
df["rating"] = df["time"]
df = df.drop("time", axis = 1)

In [3]:
user_group_dfs = list(df.groupby('user')['item'])
first_row = True
user_neg_dfs = pd.DataFrame()
items = set(df["item"])
for u, u_items in tqdm(user_group_dfs):
    u_items = set(u_items)
    num_negative = len(u_items)
    i_user_neg_item = np.random.choice(list(items - u_items), num_negative, replace=False)
    
    i_user_neg_df = pd.DataFrame({'user': [u]*num_negative, 'item': i_user_neg_item, 'rating': [0]*num_negative})
    if first_row == True:
        user_neg_dfs = i_user_neg_df
        first_row = False
    else:
        user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)
        
joined_rating_df = pd.concat([df, user_neg_dfs], axis = 0, sort=False)

100%|██████████| 31360/31360 [21:22<00:00, 24.45it/s]


In [4]:
ease = EASE()
ease.fit(joined_rating_df)

In [5]:
answer = ease.predict(df, df["user"], df["item"],10)
answer = answer.drop("score", axis = 1)
answer.to_csv("./submission2.csv", index = False)

100%|██████████| 31360/31360 [01:14<00:00, 421.46it/s]


In [38]:
ease.item_enc.transform(df["item"])

array([2505,  109,  319, ..., 5768, 4791, 4363])